In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [2]:
from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

In [3]:
RESET = False

infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()

math_article_seeder = infrastructure_container.math_article_seeder()
math_expression_seeder = infrastructure_container.math_expression_seeder()
math_expression_classification_repository = (
    infrastructure_container.math_expression_classification_seeder()
)
math_article_seeder.seed(reset=RESET)
await math_expression_seeder.seed(reset=RESET)
await math_expression_classification_repository.seed(reset=RESET)

math_article_repository = infrastructure_container.math_article_repository()
math_expression_repository = infrastructure_container.math_expression_repository()
math_expression_classification_repository = (
    infrastructure_container.math_expression_classification_repository()
)

In [ ]:
infrastructure_container.wire(
    modules=[__name__]
)  # NOTE not needed if using objects from container directly (without provide and inject)

In [ ]:
from dependency_injector.wiring import Provide, inject

from math_rag.infrastructure.seeders.objects import MathArticleSeeder


@inject
def run_seeder(
    seeder: MathArticleSeeder = Provide[InfrastructureContainer.math_article_seeder],
):
    seeder.seed()

In [ ]:
# NOTE this code doesnt require wiring
math_article_seeder = infrastructure_container.math_article_seeder()
math_article_seeder.seed()